# Note Cards to Digital Notes

I have a lot of physical cue cards that I need to place in my obsidian notes. So I created a script that uses Google's Vision API and OpenAI's API to extract the text from the pictures of my Note cards and place the text in a text file. 

In [1]:
from google.cloud import vision
from google.cloud.vision_v1 import types
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "serviceAccountCred.json"

In [2]:
from google.cloud import vision
import io

def extract_text(image_path):
    client = vision.ImageAnnotatorClient()

    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations

    if response.error.message:
        raise Exception(response.error.message)

    # Extracted text
    return texts[0].description if texts else ""


In [3]:
import openai

openai.api_key = os.environ.get('OPENAI_API_KEY')
client = openai.OpenAI()

def process_text(extracted_text):

    messages = [
            {"role": "system", "content": "There are sometimes titles and subtitles in the first few lines, create a new line when this occurs. Be sure to eliminate any other new lines"},
            {"role": "system", "content": "If the first set of letters are 'DG' or similar Convert to label '*Daring Greatly*', If the first set of letters are 'TGOI' or similar Convert to label '*The gift of imperfection*', if it is neither use the previous label"},
            {"role": "user", "content": f"Process this extracted text and stay as close to the orginal text as possible as well as making the text grammtically correct: {extracted_text}"}
        ]
        
    # Generate a response from the model
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        response_format={"type": "text"},
    )
    
    # Return the assistant's message (the cover letter)
    return response.choices[0].message.content



In [4]:

# Define the folder path
folder_path = r"Data/Note Card Pictures"

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    # Making sure to only process the .jpg images
    if filename.lower().endswith(".jpg"):
        # Construct the full path to the image
        image_path = os.path.join(folder_path, filename)

        extracted_text = extract_text(image_path)
        processed_text = process_text(extracted_text)
        
        # save to the text file, so I can copy it later into my obsidian notes
        with open("BreneBrownNotes.txt", "a") as f:
            f.write(processed_text + "\n\n\n\n")  # Add a newline between paragraphs for readability

print("Processing complete. View the BreneBrownNotes.txt.")


Processing complete. View the BreneBrownNotes.txt
